In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [6]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.mixture import GaussianMixture
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

<IPython.core.display.Javascript object>

In [3]:
from sqlalchemy import create_engine

postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "heartdisease"

engine = create_engine(
    f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"
)

sql = """
    SELECT * FROM heartdisease
"""

heartdisease_df = pd.read_sql(sql, engine)
engine.dispose()

<IPython.core.display.Javascript object>

In [4]:
# Define the features and the outcome
X = heartdisease_df.iloc[:, :13]
y = heartdisease_df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace="?", value=0)

# Binarize y so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

<IPython.core.display.Javascript object>

1. Apply GMM to the heart disease dataset by setting n_components=2. Get ARI and silhouette scores for your solution and compare it with those of the k-means and hierarchical clustering solutions that you implemented in the previous checkpoint assignments. Which algorithm performs best?

In [10]:
n_components = 2
gmm = GaussianMixture(n_components=n_components)
y_pred = gmm.fit_predict(X_std)

print(
    f"\nARI score for {n_components} components: {metrics.adjusted_rand_score(y, y_pred)}\n"
    f"Silhouette coefficient for {n_components} components: {metrics.silhouette_score(X_std, y_pred, metric='euclidean')}"
)


ARI score for 2 components: 0.4207322145049338
Silhouette coefficient for 2 components: 0.16118591340148433


<IPython.core.display.Javascript object>

In [11]:
k = 2
km = KMeans(n_clusters=k)
y_pred = km.fit_predict(X_std)

print(
    f"\nARI score for kmeans, k = {n_components}: {metrics.adjusted_rand_score(y, y_pred)}\n"
    f"Silhouette coefficient for kmeans, k = {n_components}: {metrics.silhouette_score(X_std, y_pred, metric='euclidean')}"
)


ARI score for kmeans, k = 2: 0.4380857727169879
Silhouette coefficient for kmeans, k = 2: 0.17530682286260937


<IPython.core.display.Javascript object>

In [12]:
links = ["complete", "average", "ward"]
for link in links:
    agg_cluster = AgglomerativeClustering(n_clusters=2, linkage=link)
    y_pred = agg_cluster.fit_predict(X_std)
    print(
        f"\nARI score for Agglomerative Clustering with {link} linkage method: {metrics.adjusted_rand_score(y, y_pred)}\n"
        f"Silhouette coefficient for Agglomerative Clustering with {link} linkage method: {metrics.silhouette_score(X_std, y_pred, metric='euclidean')}"
    )


ARI score for Agglomerative Clustering with complete linkage method: -0.0010061512759301477
Silhouette coefficient for Agglomerative Clustering with complete linkage method: 0.34836188921330785

ARI score for Agglomerative Clustering with average linkage method: -0.0010061512759301477
Silhouette coefficient for Agglomerative Clustering with average linkage method: 0.34836188921330785

ARI score for Agglomerative Clustering with ward linkage method: 0.146129913123814
Silhouette coefficient for Agglomerative Clustering with ward linkage method: 0.1387197366557222


<IPython.core.display.Javascript object>

2. GMM implementation of scikit-learn has a parameter called covariance_type. This parameter determines the type of covariance parameters to use. There are four types that you can specify:

* full: This is the default. Each component has its own general covariance matrix.
* tied: All components share the same general covariance matrix.
* diag: Each component has its own diagonal covariance matrix.
* spherical: Each component has its own single variance.

Try all of these. Which one performs best in terms of ARI and silhouette scores?

In [13]:
n_components = 2
covariance_types = ["full", "tied", "diag", "spherical"]

for cov_type in covariance_types:
    gmm = GaussianMixture(n_components=n_components, covariance_type=cov_type)
    y_pred = gmm.fit_predict(X_std)

    print(
        f"\nARI score with {cov_type} covariance: {metrics.adjusted_rand_score(y, y_pred)}\n"
        f"Silhouette coefficient with {cov_type} covariance: {metrics.silhouette_score(X_std, y_pred, metric='euclidean')}"
    )


ARI score with full covariance: 0.4207322145049338
Silhouette coefficient with full covariance: 0.16118591340148433

ARI score with tied covariance: 0.18389186035089963
Silhouette coefficient with tied covariance: 0.13628813153331445

ARI score with diag covariance: 0.3870024156200561
Silhouette coefficient with diag covariance: 0.1604139815113049

ARI score with spherical covariance: 0.20765243525722465
Silhouette coefficient with spherical covariance: 0.12468753110276876


<IPython.core.display.Javascript object>

In [ ]:
# full covariance looks like it performs best